In [ ]:
!pip install pyngrok

In [ ]:
import getpass

from pyngrok import ngrok, conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()

# Open a TCP ngrok tunnel to the SSH server
connection_string = ngrok.connect("22", "tcp").public_url

ssh_url, port = connection_string.strip("tcp://").split(":")
print(f" * ngrok tunnel available, access with `ssh root@{ssh_url} -p{port}`")

In [ ]:
!pip install line-bot-sdk

In [ ]:
!pip install google-genai

In [ ]:
from flask import Flask, request, abort

# === 啟用ngrok並顯示公開網址 ===
from pyngrok import ngrok

port = "5000"

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")
# ==========

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

# 匯入Get Content API所需的程式庫
from linebot.v3.messaging import MessagingApiBlob
import base64

# 匯入Loading API所需的程式庫
from linebot.v3.messaging import ShowLoadingAnimationRequest

from linebot.v3.messaging import (
    FlexContainer,
    FlexMessage
)

# 匯入Gemini API所需的程式庫
from google import genai
from google.genai import types
import json

# 實作生成函式
def generate(imageData):
    client = genai.Client(
        api_key="GEMINI_API_KEY"
    )

    model = "gemini-2.0-flash"

    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_bytes(
                    data=imageData,
                    mime_type="image/jpeg",
                ),
                types.Part.from_text(text="""你是美食評論家，請根據這張圖片（無論是否為食物）給予一段正面的、鼓勵的話語，像在廟宇抽到的上上籤一樣，最多四句，不超過50個字。請輸出JSON純文字，格式是{\"comment\":string}，全部合併在同一行就好，我要直接套用到程式裡，請不要有斷行（\\n）等額外字元。"""),
            ],
        )
    ]

    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        top_k=40,
        max_output_tokens=8192,
        response_mime_type="application/json",
    )

    response = ""

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        response += chunk.text

    return json.loads(response)["comment"]

app = Flask(__name__)

configuration = Configuration(access_token='YOUR_CHANNEL_ACCESS_TOKEN')
handler = WebhookHandler('YOUR_CHANNEL_SECRET')

@app.route("/", methods=['POST']) # 刪掉callback，Webhook網址短一點
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent)
def handle_message(event):
    if (event.message.type == "image"):
        with ApiClient(configuration) as api_client:
            line_bot_api = MessagingApi(api_client)

            # 顯示載入動畫
            line_bot_api.show_loading_animation(ShowLoadingAnimationRequest(chatId=event.source.user_id, loadingSeconds=30))

            # 取得圖片內容並進行Base64編碼
            line_bot_api_blob = MessagingApiBlob(api_client)
            receivedImage = base64.b64encode(line_bot_api_blob.get_message_content(event.message.id)).decode('utf-8')

            # 組合Flex彈性訊息
            flex = {"type":"bubble","size":"giga","body":{"type":"box","layout":"vertical","contents":[{"type":"text","text":"針對這道美食，我想說的是…","size":"xl","align":"center","weight":"bold"},{"type":"separator","color":"#000000","margin":"md"},{"type":"box","layout":"horizontal","contents":[{"type":"image","url":"https://chibu.app/reviewer.jpg","size":"full","aspectRatio":"1:1","flex":1},{"type":"text","text":generate(receivedImage),"gravity":"center","align":"center","adjustMode":"shrink-to-fit","flex":2,"size":"lg","wrap":True,"margin":"md"}],"margin":"md"}]}}

            # 以Flex彈性訊息回覆
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[FlexMessage(alt_text="關於這道美食，我想說的是…", contents=FlexContainer.from_json(json.dumps(flex)))]
                )
            )
    else:
        with ApiClient(configuration) as api_client:
            line_bot_api = MessagingApi(api_client)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="請上傳一張美食照片給我，我可以幫您寫評論喔！")]
                )
            )

if __name__ == "__main__":
    app.run()